<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
import random
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import math
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler 

In [322]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)
def sigmoid(x):
  output = [0] * len(x)
  for index, item in enumerate(x): 
    output[index] = 1.0 if item > 0.0 else 0.0 
  return output
def relu(x):
  output = [0] * len(x)
  for index, item in enumerate(x): 
    output[index] = item if item > 0.0 else item * 0.05
  return output

In [220]:
output_labels = [0.0, 1.0, 2.0]

In [333]:
class Layer():
  def __init__(self, name, units, input_dim):
    self.name = name
    self.w = [[random.uniform(0,1) for _ in range(input_dim)] for _ in range(units)]
    self.b = [0 for _ in range(units)]
    self.output = [bias for bias in self.b]
    self.next_layer = None
    self.previous_layer = None
    self.error = [0 for _ in range(units)]
  def set_previous_layer(self, previous_layer):
    self.previous_layer = previous_layer
  def set_next_layer(self, next_layer):
    self.next_layer = next_layer
  def calculate_output(self, inputs, activation=sigmoid):
    self.output = [bias for bias in self.b]
    self.output += np.dot(self.w, inputs)
    self.output = activation(self.output)
    return self.output 
  def update_error(self, expected_output):
    if self.next_layer == None:
      inputs = self.previous_layer.output
      # total_error = sum([(exp_output-self.output[i])**2 for i, exp_output in enumerate(expected_output)])
      # total_error = log_loss(expected_output, [self.output], labels=output_labels)
      total_error = categorical_cross_entropy(self.output, expected_output)
      for row_index, col in enumerate(self.w):
        self.error[row_index] = total_error * self.output[row_index] * (1-self.output[row_index]) 
    else:
      weights = self.next_layer.w
      next_layer_error = self.next_layer.error
      for output_idx, output in enumerate(self.output):
        print(output)
        self.error[output_idx] = sum([weight[output_idx] * next_layer_error[weight_idx] * output * (1-output) for weight_idx, weight in enumerate(weights)])
  def update_weights(self, learning_rate, row):
    if self.next_layer == None:
      inputs = self.previous_layer.output
      for row_index, col in enumerate(self.w):
        for col_index, weight in enumerate(col):
          self.w[row_index][col_index] = self.w[row_index][col_index] + (learning_rate * self.error[row_index] * inputs[col_index])
          self.b[row_index] += learning_rate * self.error[row_index]
    else:
      inputs = row
      for row_index, col in enumerate(self.w):
        for col_index, weight in enumerate(col):
          self.w[row_index][col_index] = self.w[row_index][col_index] + (learning_rate * self.error[row_index] * inputs[col_index])
          self.b[row_index] += learning_rate * self.error[row_index]
  
    

In [308]:
def reset_layers():
  layer2 = Layer("Output", 3, 3)
  layer1 = Layer("Hidden1", 3, 2)
  layer2.set_previous_layer(layer1)
  layer1.set_next_layer(layer2)
  layers = [layer1, layer2]
  return layers

In [146]:
iris = load_iris()

iris_columns = ['sepal_len', 'sepal_width', 'petal_length', 'petal_width', 'target']
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris_columns)

In [298]:
from sklearn.model_selection import train_test_split
# X = df[['sepal_len', 'sepal_width', 'petal_length', 'petal_width']]
X = df[['sepal_width', 'petal_length']]
y = df[['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state = 10)


# scaler_training = StandardScaler() 
# scaler_training.fit(X_train)
# scaled_values_training = scaler_training.transform(X_train)
# scaled_values_testing = scaler_training.transform(X_test)

# X_train = pd.DataFrame(scaled_values_training, columns=[*X_train.columns.values])
# X_test = pd.DataFrame(scaled_values_testing, columns=[*X_test.columns.values])

# X_train['bias'] = 1
# X_test['bias'] = 1
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)


In [299]:
X_train

,sepal_width,petal_length
0,2.9,4.6
1,2.9,4.3
2,3.0,5.8
3,2.8,5.1
4,2.5,5.0
...,...,...
115,2.5,5.0
116,2.9,3.6
117,4.4,1.5
118,3.2,6.0


In [338]:
import random
random.seed(34)
selected_rows = random.sample(range(0,119),   10)

In [149]:
lb = preprocessing.LabelBinarizer()
lb.fit([0,1,2])
lb.transform([0])


array([[1, 0, 0]])

In [150]:
def categorical_cross_entropy(actual, predicted):
	sum_score = 0.0
	for i in range(len(actual)):
			sum_score += actual[i] * math.log(1e-15 + predicted[i])
	mean_sum_score = (1.0 / len(actual)) * sum_score
	return -mean_sum_score

In [341]:
def train_weights_using_mini_batch_gradient_descent(X_train, y_train, num_epochs, learning_rate, layers):
  # weights for all the features 
  index = 0;
  
  mini_batch = X_train.loc[selected_rows, :]
  y_train = y_train.loc[selected_rows, :]
  while index < num_epochs:
    sum_error = 0
    print("============================================")
    print("=================Iteration %d==============="%(index+1))
    print("============================================")
    for rowIndex, row in mini_batch.iterrows():
      # print(row)
      output = row
      for layer_idx, layer in enumerate(layers):
        if layer_idx == len(layers) - 1:
          output = layer.calculate_output(output, softmax)
        else:
          output = layer.calculate_output(output, relu)

      expected_output = lb.transform([y_train.loc[rowIndex,'target']])[0]
      sum_error += sum([(exp_output-output[i])**2 for i, exp_output in enumerate(expected_output)])
      for layer_idx, layer in enumerate(reversed(layers)):
        # layer.update_error([y_train.loc[rowIndex,'target']])
        layer.update_error(expected_output)
      print("error of output layer", layers[1].error)
      print("error of hidden layer", layers[0].error)  
      for layer_idx, layer in enumerate(reversed(layers)):
        layer.update_weights(learning_rate, row)
      # error = log_loss([y_train.loc[rowIndex,'target']], [output], labels=[0.0, 1.0, 2.0])
    print("Entropy based loss: %f"% (sum_error))
      # if error != 0:
      #   layer1_error = layers[1].update_weights(error, layers[0].output, learning_rate)
      #   layers[0].update_weights(layer1_error, row, learning_rate)
    print("New Weights of output layer", layers[1].w)
    print("New Weights of hidden layer", layers[0].w)
    # for layer_idx, layer in enumerate(layers):
    #   layer.update_weights()
    index = index + 1
  # df_plot = pd.DataFrame(df_plot, columns=['mse', 'epoch'])
  return

layers = reset_layers()
print("Creating model using training data")
train_weights_using_mini_batch_gradient_descent(X_train, y_train, 10, 0.5, layers)

Creating model using training data
=================Iteration 1===============
3.8894648932609908
4.155187012494743
3.7898532744437294
error of output layer [0.05209496214113262, 0.09239967751794666, 0.045077465658441916]
error of hidden layer [-0.924075000068667, -0.8584776336890476, -1.5093839020724384]
-0.40723919817466525
-0.3433495431769678
-0.8234527517991035
error of output layer [1.7693436286871802, 1.338666933343476, 1.697524444892754]
error of hidden layer [-1.5172439754973923, -0.8369635748002537, -5.8390712558725095]
-0.9314955841883636
-0.564435611161566
-3.306933206097113
error of output layer [0.7557494110407533, 0.09115062068687452, 0.7173251205541585]
error of hidden layer [-0.5357466039249934, 0.051367427788999435, -0.4125112120237271]
-2.0306564403453
-1.0682053726195073
-6.259675881190513
error of output layer [1.0830121924627834, 5.253338511948132e-06, 1.08300822133691]
error of hidden layer [2.225941146048727, 1.3384653310653793, 120.75827894693508]
-0.05637759815

In [336]:
for rowIndex, row in X_test.iterrows():
  # print(rowIndex)
  output = row
  for layer_idx, layer in enumerate(layers):
      if layer_idx == len(layers) - 1:
        output = layer.calculate_output(output, softmax)
      else:
        output = layer.calculate_output(output, relu)
  # print(output)
  print("Predicted: %d, Actual %d"%(np.argmax(output), y_test.loc[rowIndex, 'target']))

Predicted: 1, Actual 1
Predicted: 1, Actual 2
Predicted: 1, Actual 0
Predicted: 1, Actual 1
Predicted: 1, Actual 0
Predicted: 1, Actual 1
Predicted: 1, Actual 1
Predicted: 1, Actual 1
Predicted: 1, Actual 0
Predicted: 1, Actual 1
Predicted: 1, Actual 1
Predicted: 1, Actual 2
Predicted: 1, Actual 1
Predicted: 1, Actual 0
Predicted: 1, Actual 0
Predicted: 1, Actual 2
Predicted: 1, Actual 1
Predicted: 1, Actual 0
Predicted: 1, Actual 0
Predicted: 1, Actual 0
Predicted: 1, Actual 2
Predicted: 1, Actual 2
Predicted: 1, Actual 2
Predicted: 1, Actual 0
Predicted: 1, Actual 1
Predicted: 1, Actual 0
Predicted: 1, Actual 1
Predicted: 1, Actual 1
Predicted: 1, Actual 1
Predicted: 1, Actual 2
